In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats import norm
import math

In [2]:
rates = pd.read_csv('./InterestRates.csv')
#rates['key'] = rates['pricedate'].str.cat(rates['maturity'] , sep =", ")
rates['pricedate_str'] = rates['pricedate']
rates['pricedate'] = pd.to_datetime(rates.pricedate, format='%Y-%m-%d', errors='ignore')
#print (rates.dtypes)
#rates.info()
rates[['number1','month']] = rates.maturity.str.split("M",expand=True,)
rates[['number2','year']] = rates.maturity.str.split("Y",expand=True,)

#So, I want to place number with pfv where pfv is not null, how can I achieve that?
rates['number'] = np.where(~rates['year'].isnull(),rates['number2'],rates['number1'])
rates['number'] = pd.to_numeric(rates['number'])
rates['MorY'] = np.where(~rates['year'].isnull(),'Y','M')

rates = rates.drop(columns=['number1','number2','month','year'])
rates['months'] = np.where(rates['MorY']=='Y',rates['number']*12,rates['number']*1)

#generating key2
#total_rows['ColumnID'] = total_rows['ColumnID'].astype(str)
rates['months_str'] = rates['months'].astype(str)
rates['key2'] = rates['pricedate_str'].str.cat(rates['months_str'], sep=',')

rates.info()
rates.head()

#for the risk free rates I am thinking on using the smallest one, 1 month
#rates_1m = rates[rates['maturity']=='1M']
#rates_1m.info()
#rates_1m.head()

#len(rates.key.unique())
#len(rates.maturity.unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7085 entries, 0 to 7084
Data columns (total 9 columns):
pricedate        7085 non-null datetime64[ns]
maturity         7085 non-null object
bidrate          7085 non-null float64
pricedate_str    7085 non-null object
number           7085 non-null int64
MorY             7085 non-null object
months           7085 non-null int64
months_str       7085 non-null object
key2             7085 non-null object
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 498.2+ KB


,pricedate,maturity,bidrate,pricedate_str,number,MorY,months,months_str,key2
0,2020-03-02,100M,1.463333,2020-03-02,100,M,100,100,"2020-03-02,100"
1,2020-03-02,101M,1.466667,2020-03-02,101,M,101,101,"2020-03-02,101"
2,2020-03-02,102M,1.470001,2020-03-02,102,M,102,102,"2020-03-02,102"
3,2020-03-02,103M,1.478335,2020-03-02,103,M,103,103,"2020-03-02,103"
4,2020-03-02,104M,1.486669,2020-03-02,104,M,104,104,"2020-03-02,104"


In [5]:
prices = pd.read_csv('./PricesFile1.csv')
prices['pricedate_str'] = prices['pricedate']
prices['delivdate_str'] = prices['delivdate']
prices['key'] = prices['pricedate_str'].str.cat(prices['delivdate_str'], sep=',')

#drop other variables
prices = prices[prices['priceindex']=='NYMEX Natural Gas']
#'AECO NIT MONTHLY (7A) US$' 'AECO NIT MONTHLY (7A) US$ FWD'

#fixing dates
#pricedate and delivdate
prices['pricedate'] = pd.to_datetime(prices.pricedate, format='%Y-%m-%d', errors='ignore')
prices['delivdate'] = pd.to_datetime(prices.delivdate, format='%Y-%m-%d', errors='ignore')
#prices.info()
#negative prices
neg_prices = prices[prices['price']<0]


#calculating duration ****QUESTION Should we drop negatives according to scott answer?
prices['duration'] = prices['delivdate'] - prices['pricedate']
prices['duration_months'] = round(prices['duration'].dt.days/30).astype(int)

#generating key2
prices['duration_months'][prices['duration_months']==0] = 1
prices['duration_months_str'] = prices['duration_months'].astype(str)
prices['key2'] = prices['pricedate_str'].str.cat(prices['duration_months_str'], sep=',')

#prices.info()

#drop negative duration  **63 elements dropped
prices_drop = prices[prices['duration'].dt.days<0]
prices = prices[prices['duration'].dt.days>=0]

prices = prices.drop(columns=['pricedate_str','delivdate_str'])

#prices[(prices['pricedate']=='2020-04-20')& (prices['delivdate']=='2020-05-01')]

#Day of negative
#prices[prices['pricedate']=='2020-04-20']

#Checking if prices are unique for same dates
#len(prices.key.unique())

#prices.priceindex.unique()
#neg_prices.info()
#neg_prices.head()
#prices.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
NG_imvl20=pd.read_csv("./NG_ImpliedVols2020.csv")
NG_imvl21=pd.read_csv("./NG_ImpliedVols2021.csv")
NG_imvl22=pd.read_csv("./NG_ImpliedVols2022.csv")

#Append
frames=[NG_imvl20,NG_imvl21,NG_imvl22]
NG_imvl = pd.concat(frames)

#generating key
NG_imvl['volatilitydate_str'] = NG_imvl['volatilitydate']
NG_imvl['begtime_str'] = NG_imvl['begtime']
NG_imvl['key'] = NG_imvl['volatilitydate_str'].str.cat(NG_imvl['begtime_str'], sep=',')

#fixing dates
#volatilitydate and begtime
NG_imvl['volatilitydate'] = pd.to_datetime(NG_imvl.volatilitydate, format='%Y-%m-%d', errors='ignore')
NG_imvl['begtime'] = pd.to_datetime(NG_imvl.begtime, format='%Y-%m-%d', errors='ignore')

#calculating duration ****QUESTION Should we drop negatives according to scott answer?
NG_imvl['duration'] = NG_imvl['begtime'] - NG_imvl['volatilitydate']
NG_imvl['duration_months'] = round(NG_imvl['duration'].dt.days/30,0).astype(int)

#Take previous day volatility

#the negative day
neg_day = NG_imvl[(NG_imvl['volatilitydate']=='2020-04-20') & (NG_imvl['begtime']=='2020-05-01')]


#drop negative duration  **No elements dropped
NG_imvl = NG_imvl[NG_imvl['duration_months']>=0]
NG_imvl = NG_imvl[NG_imvl['duration_months']>=0]
#Checking if NG_imvl are unique for same dates
len(NG_imvl.key.unique())
#NG_imvl.info()

###generating key2
#NG_imvl['duration_months'][NG_imvl['duration_months']==0] = 1
#NG_imvl['duration_months_str'] = NG_imvl['duration_months'].astype(str)
#NG_imvl['key2'] = NG_imvl['volatilitydate_str'].str.cat(NG_imvl['duration_months_str'], sep=',')

#####Volatility
#key                1198 non-null object
#strikeprice        1198 non-null float64
#volatility         1198 non-null float64
#volatilityindex    1198 non-null object
NG_imvl = NG_imvl.drop(columns=['volatilitydate','begtime','duration','duration_months','volatilitydate_str','begtime_str'])

#Left join
master_NG = pd.merge(prices,NG_imvl,on='key',how='left')

#Replace NA's by the previous observation 
na_master_NG = master_NG[np.isnan(master_NG['volatility'])]
na_master_NG.reset_index(drop=True, inplace=True)

for i in range(len(na_master_NG)):
    first_date = na_master_NG['pricedate'][i]
    last_date = na_master_NG['delivdate'][i]
    sub_set = master_NG[(master_NG['pricedate']<=first_date)& (master_NG['delivdate']==last_date)].sort_values(by='pricedate', ascending=False)
    sub_set.reset_index(drop=True, inplace=True)
    j=0
    while np.isnan(sub_set['volatility'][j]) and sub_set['pricedate'][j] > datetime.strptime('2020-03-02', '%Y-%m-%d'):
        j = j+1    # update counter
    
    if j>0 and sub_set['pricedate'][j] > datetime.strptime('2020-03-02', '%Y-%m-%d'):
        index_save = master_NG.index[(master_NG['pricedate']==first_date)& (master_NG['delivdate']==last_date)][0]
        master_NG['volatilityindex'].at[index_save]=(sub_set['volatilityindex'][j+1])
        master_NG['strikeprice'].at[index_save]=(sub_set['strikeprice'][j+1])
        master_NG['volatility'].at[index_save]=(sub_set['volatility'][j+1])
        #master_NG['key2'].at[index_save]=(sub_set['key2'][j+1])
        del(index_save)

    del(first_date,last_date,sub_set)

master_NG = master_NG.dropna()

#master_NG[(master_NG['pricedate']=='2020-04-20')& (master_NG['delivdate']=='2020-05-01')]
            
#len(master_NG[np.isnan(master_NG['volatility'])])

master_NG.info()
#master_NG.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 152833 entries, 0 to 152859
Data columns (total 12 columns):
priceindex             152833 non-null object
pricedate              152833 non-null datetime64[ns]
delivdate              152833 non-null datetime64[ns]
price                  152833 non-null float64
key                    152833 non-null object
duration               152833 non-null timedelta64[ns]
duration_months        152833 non-null int64
duration_months_str    152833 non-null object
key2                   152833 non-null object
volatilityindex        152833 non-null object
strikeprice            152833 non-null float64
volatility             152833 non-null float64
dtypes: datetime64[ns](2), float64(3), int64(1), object(5), timedelta64[ns](1)
memory usage: 15.2+ MB


In [7]:
#Merging Rates
rates2 = rates.drop(columns=['pricedate','pricedate_str','number','MorY','months','months_str'])

#Left join
master_NG = pd.merge(master_NG,rates2,on='key2',how='left')
master_NG[(master_NG['pricedate']=='2020-04-20')& (master_NG['delivdate']=='2020-05-01')]

master_NG.to_csv ('master_NG_complete.csv', index = False, header=True)
master_NG.info()
#master_NG.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152833 entries, 0 to 152832
Data columns (total 14 columns):
priceindex             152833 non-null object
pricedate              152833 non-null datetime64[ns]
delivdate              152833 non-null datetime64[ns]
price                  152833 non-null float64
key                    152833 non-null object
duration               152833 non-null timedelta64[ns]
duration_months        152833 non-null int64
duration_months_str    152833 non-null object
key2                   152833 non-null object
volatilityindex        152833 non-null object
strikeprice            152833 non-null float64
volatility             152833 non-null float64
maturity               152833 non-null object
bidrate                152833 non-null float64
dtypes: datetime64[ns](2), float64(4), int64(1), object(6), timedelta64[ns](1)
memory usage: 17.5+ MB


In [8]:
#Black 76


# t = pricedate          1198 non-null datetime64[ns]
# T = delivdate          1198 non-null datetime64[ns]
# F(t,T) = price              1198 non-null float64
# T - t = duration_n         1198 non-null int64
# K = strikeprice        1198 non-null float64
# sigma = volatility         1198 non-null float64
# r = bidrate            1198 non-null float64

master_NG.rename(columns={'pricedate': 't', 'delivdate': 'T', 'price': 'F', 'strikeprice': 'K', 'volatility': 'sigma', 'bidrate': 'r'}, inplace=True)

master_NG['T_t'] = master_NG['duration'].dt.days

#FIX A Te
#For creating the loop, check if Te_t < T, if true, calculate the call value
#Te_t = 15/360

master_NG = master_NG[master_NG['T_t']>30]
Te_t = (master_NG['T_t']-30)/360

#Calculating call value
Nd1 = norm.cdf((np.log(master_NG['F']/master_NG['K']) + (1/2 * master_NG['sigma']**2 * Te_t))/(master_NG['sigma'] * np.sqrt(Te_t)))
Nd2 = norm.cdf((np.log(master_NG['F']/master_NG['K']) - (1/2 * master_NG['sigma']**2 * Te_t))/(master_NG['sigma'] * np.sqrt(Te_t)))

master_NG['call'] = np.exp(-(master_NG['r']/100)*Te_t)*(master_NG['F']*Nd1 - master_NG['K']*Nd2)

Nd1_put = norm.cdf(-(np.log(master_NG['F']/master_NG['K']) + (1/2 * master_NG['sigma']**2 * Te_t))/(master_NG['sigma'] * np.sqrt(Te_t)))
Nd2_put = norm.cdf(-(np.log(master_NG['F']/master_NG['K']) - (1/2 * master_NG['sigma']**2 * Te_t))/(master_NG['sigma'] * np.sqrt(Te_t)))

master_NG['put'] = np.exp(-(master_NG['r']/100)*Te_t)*(master_NG['K']*Nd2_put - master_NG['F']*Nd1_put )

#master_NG[(master_NG['t']=='2020-04-20')& (master_NG['T']=='2020-05-01')]
master_NG.info()
#master_NG.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149436 entries, 49 to 152832
Data columns (total 17 columns):
priceindex             149436 non-null object
t                      149436 non-null datetime64[ns]
T                      149436 non-null datetime64[ns]
F                      149436 non-null float64
key                    149436 non-null object
duration               149436 non-null timedelta64[ns]
duration_months        149436 non-null int64
duration_months_str    149436 non-null object
key2                   149436 non-null object
volatilityindex        149436 non-null object
K                      149436 non-null float64
sigma                  149436 non-null float64
maturity               149436 non-null object
r                      149436 non-null float64
T_t                    149436 non-null int64
call                   149436 non-null float64
put                    149436 non-null float64
dtypes: datetime64[ns](2), float64(6), int64(2), object(6), timedelta64[ns](1

In [10]:
master_NG.to_csv ('master_NG_call_put_black76.csv', index = False, header=True)
master_NG.head()

,priceindex,t,T,F,key,duration,duration_months,duration_months_str,key2,volatilityindex,K,sigma,maturity,r,T_t,call,put
49,NYMEX Natural Gas,2020-03-02,2020-05-01,1.796,"2020-03-02,2020-05-01",60 days,2,2,"2020-03-02,2",V_NYMEX Natural Gas,1.01,0.605,2M,0.995003,60,0.785379,0.000030
50,NYMEX Natural Gas,2020-03-02,2020-05-01,1.796,"2020-03-02,2020-05-01",60 days,2,2,"2020-03-02,2",V_NYMEX Natural Gas,1.06,0.590,2M,0.995003,60,0.735453,0.000063
51,NYMEX Natural Gas,2020-03-02,2020-05-01,1.796,"2020-03-02,2020-05-01",60 days,2,2,"2020-03-02,2",V_NYMEX Natural Gas,1.11,0.575,2M,0.995003,60,0.685558,0.000127
52,NYMEX Natural Gas,2020-03-02,2020-05-01,1.796,"2020-03-02,2020-05-01",60 days,2,2,"2020-03-02,2",V_NYMEX Natural Gas,1.15,0.609,2M,0.995003,60,0.645916,0.000451
53,NYMEX Natural Gas,2020-03-02,2020-05-01,1.796,"2020-03-02,2020-05-01",60 days,2,2,"2020-03-02,2",V_NYMEX Natural Gas,1.20,0.589,2M,0.995003,60,0.596243,0.000737
